In [ ]:
# importing required libraries
import numpy as np
import torch
from torch.nn.functional import conv2d

In [ ]:
# forward propagation for convolution
def conv2d_forward(matrix, filter):
  h_x, w_x, _ = matrix.shape # getting matrix shape
  h_w, w_w, _  = filter.shape # getting filter shape
  output = np.zeros((h_x - h_w + 1, w_x - w_w + 1)) # initializing output matrix
  for i in range(len(output)): # for each pixel
    for j in range(len(output[i])):
        output[i][j] = np.sum(matrix[i:i+h_w, j:j+w_w, :] * filter) # calculate sum of hadamart product between
                                                                    # matrix batch and filter, save value in output cell
  return output

In [ ]:
# backward propagation for convolution (dL/dZ)
def conv2d_backward(upstream, filter):
  h_w, w_w, d_w  = filter.shape # getting filter shape
  padded_upstream = np.pad(upstream,((h_w - 1, h_w - 1), (w_w - 1, w_w - 1), (0, 0)), 'constant') # padding upstream matrix
  rotated_filter = np.rot90(np.rot90(filter)) # rotate filter by 180 degree
  dL_dZ = [] # initializing output
  for i in range(d_w): # for each channel
    dL_dZ.append(conv2d_forward(padded_upstream, rotated_filter[:, :, i, np.newaxis])) # adding dL/dZ
  return np.array(dL_dZ)

In [ ]:
# backward propagation for convolution (dL/dW)
def conv2d_backward_weights(weights, upstream):
  h_x, w_x, d_x  = weights.shape # getting filter shape
  dL_dZ = [] # initializing output
  for i in range(d_x): # for each channel
    dL_dZ.append(conv2d_forward(weights[:, :, i, np.newaxis], upstream)) # adding dL/dZ
  return np.array(dL_dZ)

#Testing created functions

## 1 channel

### Forward propagation

In [ ]:
# initializing input array and filter
arr1d = np.array([[[1], [2]], [[3], [4]]])

filt1d = np.array([[[10]], [[2]]])

In [ ]:
# calculating convolution forward propagation using my function
print(conv2d_forward(arr1d, filt1d))

[[16. 28.]]


In [ ]:
# initializing input array and filter as torch tensors
custom_filter = torch.tensor(filt1d.transpose(2,0,1,3), dtype=float, requires_grad=True)
input_data = torch.tensor(arr1d, dtype=float, requires_grad=True)
input_matrix = input_data.permute(2, 0, 1).unsqueeze(0)

ValueError: ignored

In [ ]:
# calculating convolution forward propagation using pytroch
output = conv2d(input_matrix, custom_filter, padding=0)
print(output)

### Backward propagation

#### $\frac{∂L}{\partial Z_{k-1}}$

In [ ]:
# calculating convolution backward propagation using my function
print(conv2d_backward(arr1d, filt1d))

In [ ]:
# initializing input array and loss as torch tensors
loss = torch.tensor(arr1d, dtype=float,  requires_grad=True)

loss = loss.permute(2, 0, 1).unsqueeze(0)

input_matrix = torch.randn(1, 1, 3, 2, dtype=float,  requires_grad=True) # dL/dZ does not depend on input matrix according to lecture slides

In [ ]:
# calculating convolution backward propagation using pytroch
output = conv2d(input_matrix, custom_filter).backward(loss)
print(input_matrix.grad.squeeze(0).numpy())

#### $\frac{∂L}{\partial W_{k}}$

In [ ]:
# calculating convolution backward propagation to find dL/dW using my function
print(conv2d_backward_weights(input_matrix.detach().squeeze(0).permute(1, 2, 0).numpy(), arr1d))

In [ ]:
# calculating convolution backward propagation to find dL/dW using pytorch
print(custom_filter.grad.numpy())

## 3 channels

### Forward propagation

In [ ]:
# initializing input array and filter
arr3d = np.array([
    [[1, 2, 3], [4, 5, 6]],
     [[7, 8, 9], [10, 11, 12]]
    ])


filt3d = np.array([[[1, 2 ,3]], [[4, 5, 6]]])

In [ ]:
# calculating convolution forward propagation using my function
conv2d_forward(arr3d, filt3d)

array([[136., 199.]])

In [ ]:
# initializing input array and filter as torch tensors
custom_filter = torch.tensor(filt3d.transpose(2,0,1)[np.newaxis, :, :,:], dtype=float, requires_grad=True)
input_data = torch.tensor(arr3d, dtype=float, requires_grad=True)
input_matrix = input_data.permute(2, 0, 1).unsqueeze(0)

In [ ]:
# calculating convolution forward propagation using pytroch
output = conv2d(input_matrix, custom_filter, padding=0)
print(output.detach().numpy())

[[[[136. 199.]]]]


### Backward propagation

#### $\frac{∂L}{\partial Z_{k-1}}$

In [ ]:
# calculating convolution backward propagation using my function
print(conv2d_backward(arr1d, filt3d))

[[[ 1.  2.]
  [ 7. 12.]
  [12. 16.]]

 [[ 2.  4.]
  [11. 18.]
  [15. 20.]]

 [[ 3.  6.]
  [15. 24.]
  [18. 24.]]]


In [ ]:
# initializing input array and loss as torch tensors
loss = torch.Tensor(arr1d)

loss = loss.permute(2, 0, 1).unsqueeze(0)

input_matrix = torch.randn(1, 3, 3, 2, dtype=float,  requires_grad=True)

In [ ]:
# calculating convolution backward propagation using pytroch
output = conv2d(input_matrix, custom_filter).backward(loss)
print(input_matrix.grad.numpy())

[[[[ 1.  2.]
   [ 7. 12.]
   [12. 16.]]

  [[ 2.  4.]
   [11. 18.]
   [15. 20.]]

  [[ 3.  6.]
   [15. 24.]
   [18. 24.]]]]


#### $\frac{∂L}{\partial W_{k}}$

In [ ]:
# calculating convolution backward propagation to find dL/dW using my function
print(conv2d_backward_weights(input_matrix.detach().squeeze(0).permute(1, 2, 0).numpy(), arr1d).shape)

(3, 2, 1)


In [ ]:
# calculating convolution backward propagation to find dL/dW using pytorch
print(custom_filter.grad.numpy())

[[[[-8.2702149 ]
   [-2.31463389]]

  [[ 2.89953331]
   [12.99310609]]

  [[ 7.42786757]
   [ 1.22592243]]]]
